In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import optuna


import tensorflow as tf
tf.random.set_seed(999)
from tensorflow.keras.regularizers import L1
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,BatchNormalization,Activation
from tensorflow.keras import regularizers



panel = pd.read_pickle('../Data/returns_chars_panel.pkl') 
macro = pd.read_pickle('../Data/macro_timeseries.pkl')

In [64]:
# combine micro and macro data
df = pd.merge(panel,macro,on='date',how='left',suffixes=['','_macro']) 

# features + targets 
X = df.drop(columns=['ret','excess_ret','rfree','permno','date']) # everything except return info and IDs
y = df['excess_ret'] 

# Split Train Test 

In [65]:
# make 20 years of training data
date = df['date']
training = (date <= '1977-03') # selects 
X_train, y_train = X.loc[training].values, y.loc[training].values 

# make 10 years of validation data
validation = (date > '1977-03') & (date <= '1987-03') 
X_val, y_val = X.loc[validation].values, y.loc[validation].values 

# make test data
test = (date > '1987-03') 
X_test, y_test = X.loc[test].values, y.loc[test].values 

In [86]:
# lamda = 1e-5
epochs = 100
# learning_rate = 0.0001
patience = 5
batch_size = 10000

# Finding baseline model for Neural Network 

In [87]:
# create the final model 
def create_nn(n_layers, input_dim, lamda, learning_rate):
    num_layers = 32 
    optimizer = Adam(learning_rate=learning_rate)
    model = Sequential()
    model.add(Dense(32, input_dim=input_dim, 
                kernel_regularizer=regularizers.L1(lamda), 
                kernel_initializer = 'he_normal'))
    model.add(BatchNormalization())

    # add extra hidden layers 
    for i in range(n_layers - 1): 
        num_layers = int(num_layers / 2)
        print(num_layers)
        model.add(Dense(num_layers,
                kernel_regularizer=regularizers.L1(lamda), 
                kernel_initializer = 'he_normal'))
        model.add(BatchNormalization())
        
    # output layer 
    model.add(Dense(1, activation='linear',
                    kernel_regularizer=regularizers.L1(0.01), 
                    kernel_initializer = 'he_normal'))

    model.compile(loss='mse', 
                optimizer=optimizer,
                metrics = ['mse']) 
    return model 


## Cross Validation for Lamda for L2 Regularization 

In [96]:
# Using Optuna to cross validate hyper parameter 
input_dim = X_train.shape[1]
n_layers = 4
def create_model(trial):

    num_layers = 32 
    # Suggest hyperparameters
    learning_rate = trial.suggest_categorical('learning_rate', [0.001, 0.01])
    l1_reg = trial.suggest_float('l1_reg', 1e-5, 1e-3, log=True)

    optimizer = Adam(learning_rate=learning_rate)
    model = Sequential()
    model.add(Dense(32, input_dim=input_dim, 
                kernel_regularizer=regularizers.L1(l1_reg), 
                kernel_initializer = 'he_normal'))
    model.add(BatchNormalization())

    # add extra hidden layers 
    for i in range(n_layers - 1): 
        num_layers = int(num_layers / 2)
        print(num_layers)
        model.add(Dense(num_layers,
                kernel_regularizer=regularizers.L1(l1_reg), 
                kernel_initializer = 'he_normal'))
        model.add(BatchNormalization())
        
    # output layer 
    model.add(Dense(1, activation='linear',
                    kernel_regularizer=regularizers.L1(0.01), 
                    kernel_initializer = 'he_normal'))

    model.compile(loss='mse', 
                optimizer=optimizer,
                metrics = ['mse']) 
    return model

# Objective function for Optuna
def objective(trial):
    model = create_model(trial)
    
    # Use early stopping
    early_stopping = EarlyStopping(patience=patience, restore_best_weights=True)

    history = model.fit(
        X_train, y_train,
        epochs,
        batch_size,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=0
    )

    # Evaluate the model
    score = model.evaluate(X_val, y_val, verbose=0)
    return score[0]

# Create a study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

# Get the best trial
best_trial = study.best_trial

print("Best trial:")
print(f"  Value: {best_trial.value}")
print("  Params: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")



[I 2024-05-24 10:18:42,183] A new study created in memory with name: no-name-4ea5b825-f68d-477d-873a-ea743ec109b7


16
8
4


[I 2024-05-24 10:19:06,213] Trial 0 finished with value: 0.026645848527550697 and parameters: {'learning_rate': 0.001, 'l1_reg': 0.0005436366706974076}. Best is trial 0 with value: 0.026645848527550697.


16
8
4


[I 2024-05-24 10:19:52,434] Trial 1 finished with value: 0.02643212489783764 and parameters: {'learning_rate': 0.001, 'l1_reg': 3.274073920839725e-05}. Best is trial 1 with value: 0.02643212489783764.


...

16
8
4


[I 2024-05-24 10:23:50,436] Trial 8 finished with value: 0.02637159638106823 and parameters: {'learning_rate': 0.001, 'l1_reg': 1.6762375688023723e-05}. Best is trial 8 with value: 0.02637159638106823.


16
8
4


[I 2024-05-24 10:24:19,073] Trial 9 finished with value: 0.02843630127608776 and parameters: {'learning_rate': 0.01, 'l1_reg': 0.000497867338072132}. Best is trial 8 with value: 0.02637159638106823.


Best trial:
  Value: 0.02637159638106823
  Params: 
    learning_rate: 0.001
    l1_reg: 1.6762375688023723e-05


# Best Hyperparameters 
## NN2 
- learning_rate = 0.001 
- l1_reg = 1.76e-05

## NN3 
- learning_rate = 0.001 
- lamda = 2.91e-05

## N4 
- learning_rate = 0.001 
- l1_reg = 1.67e-05

# 2 : Evaluate R_2_OOS 

In [ ]:
total_R_2_OOS = [] 
for i in range(1977, 1987):
    train_window = (date < datetime(i+1,1,1)) 
    test_window = (date >= datetime(i,1,1)) & (date < datetime(i+1,1,1)) 
    # print('Train', np.sum(train_window))
    # print('Test', np.sum(test_window))
    X_train_expanding, y_train_expanding = X.loc[train_window].values, y.loc[train_window].values
    X_test_expanding, y_test_expanding =  X.loc[test_window].values, y.loc[test_window].values

    lamda = 1e-5
    epochs = 100
    learning_rate = 0.0001
    patience = 5
    batch_size = 10000

    optimizer = Adam(learning_rate=learning_rate)
    model = create_model(lamda)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
    history = model.fit(X_train_expanding, y_train_expanding, 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        verbose=True,
                        validation_data=(X_val, y_val),
                        callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)])

    # Plotting the training and validation MSE
    fig, ax = plt.subplots(figsize=(10, 6))
    pd.Series(history.history['mse']).plot(ax=ax, label='training mse')
    pd.Series(history.history['val_mse']).plot(ax=ax, label='validation mse')
    ax.legend()
    plt.show()

    predictions = model.predict(X_val)
    print(predictions)
    df_predictions = pd.DataFrame(predictions, columns=['Prediction'])
    df_predictions['Actual'] = y_val
    df_predictions['dif_squared'] = (df_predictions['Prediction'] - df_predictions['Actual'])**2
    df_predictions['actual_sqaured'] = df_predictions['Actual']**2
    R_OOS = 1 - (df_predictions['dif_squared'].sum()/df_predictions['actual_sqaured'].sum()) 
    R_OOS
    

# Question 2

## Expanding Window R^2_OOS 

In [97]:
end_year = date.max()

total_R_2_OOS = [] 
model = create_nn(2, X_test.shape[1], 1.76e-05 , 0.001)
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

for i in range(1987 + 1, end_year.year):
    print(i)
    train_window = (date > datetime(1987,3,1)) & (date < datetime(i+1,1,1)) 
    test_window = (date >= datetime(i,1,1)) & (date < datetime(i+1,1,1)) 
    X_train_expanding, y_train_expanding = X.loc[train_window].values, y.loc[train_window].values
    X_test_expanding, y_test_expanding =  X.loc[test_window].values, y.loc[test_window].values

    # model.summary()
    history = model.fit(X_train_expanding, y_train_expanding, 
                        epochs=100, 
                        batch_size=batch_size, 
                        verbose=True,
                        validation_data = (X_val, y_val),
                        callbacks = [EarlyStopping(patience = patience, restore_best_weights=True)])
    predictions = model.predict(X_test_expanding)
    df_predictions = pd.DataFrame(predictions, columns=['Prediction'])
    df_predictions['Actual'] = y_test_expanding
    df_predictions['dif_squared'] = (df_predictions['Prediction'] - df_predictions['Actual'])**2
    df_predictions['actual_sqaured'] = df_predictions['Actual']**2
    R_OOS = 1 - (df_predictions['dif_squared'].sum()/df_predictions['actual_sqaured'].sum()) 
    total_R_2_OOS.append(R_OOS)


np.mean(total_R_2_OOS)
    
    


/Users/dhammatornriewcharoon/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


16
1988
Epoch 1/100
10/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.1540 - mse: 2.1005

0.012938800713457991